In [26]:
import pandas as pd
from io import StringIO
import sys

csv_data = \
'''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


### 누락된 값 삭제하기
- isnull() : Nan값을 True로 나타내고 아니면 False를 출력해 DataFrame으로 반환
- isnull().sum() : 누락된 값의 개수
- dropna(axis=0) : Nan이 하나라도 있는 행은 제거
- dropna(axis=1) : Nan이 하나라도 있는 열은 제거
- dropna(how='all') : 한 행에서 모든 값이 Nan일 때 삭제
- dropna(thresh=4) : Nan이 아닌 값이 4개 보다 작은 행을 삭제
- dropna(subset=['C']) : C열에 Nan이 있는 행을 삭제

In [12]:
df.isnull()

,A,B,C,D
0,False,False,False,False
1,False,False,True,False
2,False,False,False,True


In [14]:
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

In [16]:
df.dropna(axis=0) 

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [18]:
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [27]:
df.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [28]:
#추가 작업
csv_data = \
'''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
,,,'''

df2 = pd.read_csv(StringIO(csv_data))
df2.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0


In [29]:
df.dropna(thresh=4) 

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [30]:
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


### 누락된 값 대체하기


In [32]:
#SimpleImputer : 평균으로 대체하기
from sklearn.impute import SimpleImputer
import numpy as np

imr = SimpleImputer(missing_values=np.nan, strategy='mean') #strategy 변수 : mean, median, most_frequent
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [34]:
#FunctionTransformer : 입력 행렬의 행과 열을 바꾸어 대체할 값을 계산한 후 다시 변환된 행렬의 열과 행을 바꿈
from sklearn.preprocessing import FunctionTransformer

ftr_imr = FunctionTransformer(lambda X: imr.fit_transform(X.T).T)
imputed_data = ftr_imr.fit_transform(df.values)
imputed_data

array([[ 1.        ,  2.        ,  3.        ,  4.        ],
       [ 5.        ,  6.        ,  6.33333333,  8.        ],
       [10.        , 11.        , 12.        , 11.        ]])

In [35]:
#IterativeImputer : 상관관계를 고려하여 선형 회귀로 예측해 결측값을 대체함
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

iimr = IterativeImputer()
iimr.fit_transform(df.values)

array([[ 1.        ,  2.        ,  3.        ,  4.        ],
       [ 5.        ,  6.        ,  7.00047063,  8.        ],
       [10.        , 11.        , 12.        , 12.99964527]])

In [36]:
#KNNImputer : K-최근접 이웃 방법 사용해 누락된 값 채움
from sklearn.impute import KNNImputer

kimr = KNNImputer()
kimr.fit_transform(df.values)

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [38]:
#바로 평균값으로 대체
df.fillna(df.mean())

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,6.0


## 범주형 데이터